##### WZ: Would suggest to use the source code on github as the correct version of code as the tutorial site can be a little wrong
###### Topics: One-hot encoding, decision tree, random forest

###### Tutorial: https://subscription.packtpub.com/book/big_data_and_business_intelligence/9781789616729/6/ch06lvl1sec50/ensembling-decision-trees-random-forest
###### Source code: https://github.com/haydenliu/Python-Machine-Learning-By-Example-Second-Edition/blob/master/Chapter06/avazu_ctr_tf.py
###### Data source: https://www.kaggle.com/c/avazu-ctr-prediction/data

In [17]:
import pandas as pd
import numpy as np
n_rows = 30000
df = pd.read_csv("train.csv", nrows=n_rows)
print(df.columns)

Index(['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21'],
      dtype='object')


In [4]:
df.shape

(30000, 24)

##### The target variable is the click column.
##### Several columns do not contain much useful information and are removed

In [5]:
# wz: edited the official code. Changed from .values to .to_numpy()
X = df.drop(['click', 'id', 'hour', 'device_id', 'device_ip'], axis=1).to_numpy()
Y = df['click'].to_numpy()
print(X.shape)

(30000, 19)


Each sample has 19 predictive attributes.

Next, we need to split the data into training and testing sets. Normally, we do so by randomly picking samples. However, in our case, samples are in chronological order as indicated in the hour field. Obviously, we cannot use future samples to predict the past ones. Hence, we take the first 90% as training samples and the rest as testing samples:

In [6]:
n_train = int(n_rows * 0.9)

In [7]:
# split to training set and test set
X_train = X[:n_train]
Y_train = Y[:n_train]
X_test = X[n_train:]
Y_test = Y[n_train:]

As mentioned, decision tree models can take in categorical features. However, because the tree-based algorithms in scikit-learn (the current version is 0.20.0 as of the end of 2018) only allow numerical input, we need to transform categorical features into numerical ones. But note that in general we do not need to do so; for example, the decision tree classifier we developed from scratch earlier can directly take in categorical features.

We now transform string-based categorical features into one-hot encoded vectors using the OneHotEncoder module from scikit-learn. It basically converts a categorical feature with k possible values into k binary features. For example, the site category feature with three possible values, news, education, and sports, will be encoded into three binary features, such as is_news, is_education, and is_sports, whose values are either 1 or 0.

Initialize a OneHotEncoder object as follows:

In [8]:
# one-hot encoding process
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
X_train_enc = enc.fit_transform(X_train)
X_train_enc[0]

<1x3929 sparse matrix of type '<class 'numpy.float64'>'
	with 19 stored elements in Compressed Sparse Row format>

In [9]:
# shape before one hot encoding
print(X_train.shape)
print(X_train_enc.shape)

(27000, 19)
(27000, 3929)


In [10]:
print(X_train_enc[0])

  (0, 2)	1.0
  (0, 6)	1.0
  (0, 71)	1.0
  (0, 1002)	1.0
  (0, 1024)	1.0
  (0, 1460)	1.0
  (0, 1508)	1.0
  (0, 1529)	1.0
  (0, 2016)	1.0
  (0, 3257)	1.0
  (0, 3261)	1.0
  (0, 3361)	1.0
  (0, 3605)	1.0
  (0, 3609)	1.0
  (0, 3644)	1.0
  (0, 3735)	1.0
  (0, 3740)	1.0
  (0, 3775)	1.0
  (0, 3915)	1.0


Each converted sample is a sparse vector.

Transform the testing set using the trained one-hot encoder as follows:

In [11]:
X_test_enc = enc.transform(X_test)

Remember, we specify the handle_unknown='ignore' parameter in the one-hot encoder earlier. This is to prevent errors due to any unseen categorical values. Use the previous site category example, if there is a sample with the value movie, three converted binary features (is_news, is_education, and is_sports) all become 0s. If we do not specify ignore, an error will be raised.  

Next, we train a decision tree model using grid search, which we learned about in Chapter 5, Classifying Newsgroup Topics with a Support Vector Machine. For demonstration purposes, we only tweak the max_depth hyperparameter. Other hyperparameters, such as min_samples_split and class_weight, are also highly recommended. The classification metric should be AUC of ROC, as it is an imbalanced binary case (only 51,211 out of 300,000 training samples are clicks, that, is a 17% positive click-through rate):

##### Pick three options for the maximal depth, 3, 10, and unbounded. Initialize a decision tree model with Gini Impurity as the metric and 30 as the minimum number of samples required to split further:

In [12]:
from sklearn.tree import DecisionTreeClassifier
parameters = {'max_depth': [3, 10, None]}

decision_tree = DecisionTreeClassifier(criterion='gini',
                                       min_samples_split=30)
from sklearn.model_selection import GridSearchCV

##### As for grid search, we use three-fold (as there are enough training samples) cross-validation and select the best performing hyperparameter measured by AUC:

In [13]:
grid_search = GridSearchCV(decision_tree, parameters,
                           n_jobs=-1, cv=3, scoring='roc_auc')

##### Note n_jobs=-1 means that we use all available CPU processors:

In [14]:
grid_search.fit(X_train_enc, Y_train)
print(grid_search.best_params_)

{'max_depth': 10}


##### Use the model with the optimal parameter to predict future test cases, as follows:

In [15]:
decision_tree_best = grid_search.best_estimator_
pos_prob = decision_tree_best.predict_proba(X_test_enc)[:, 1]
from sklearn.metrics import roc_auc_score
print('The ROC AUC on testing set is: {0:.3f}'.format(roc_auc_score(Y_test, pos_prob)))

The ROC AUC on testing set is: 0.679


##### The AUC we can achieve with the optimal decision tree model is 0.72. It does not seem very high, but click-through involves many intricate human factors, which is why predicting it is not an easy problem. Although we can further optimize its hyperparameters, an AUC of 0.72 is pretty good, actually. Randomly selecting 17% of the samples to be click will generate an AUC of 0.496:

In [18]:
pos_prob = np.zeros(len(Y_test))
click_index = np.random.choice(len(Y_test),
                               int(len(Y_test) * 51211.0/300000), replace=False)
pos_prob[click_index] = 1
roc_auc_score(Y_test, pos_prob)

0.49076304543945526

##### Looking back, we can see that a decision tree is a sequence of greedy searches for the best splitting point at each step, based on the training dataset. However, this tends to cause overfitting as it is likely that the optimal points only work well for the training samples. Fortunately, ensembling is the technique to correct this, and random forest is an ensemble tree model that usually outperforms a simple decision tree.

##### The ensemble technique bagging (which stands for bootstrapaggregating) can effectively overcome overfitting. To recap, different sets of training samples are randomly drawn with replacements from the original training data; each resulting set is used to fit an individual classification model. The results of these separately trained models are then combined together through a majority vote to make the final decision.

##### Tree bagging, described in the preceding section, reduces the high variance that a decision tree model suffers from and hence, in general, performs better than a single tree. However, in some cases, where one or more features are strong indicators, individual trees are constructed largely based on these features and as a result become highly correlated. Aggregating multiple correlated trees will not make much difference. To force each tree to be uncorrelated, random forest only consider a random subset of the features when searching for the best splitting point at each node. Individual trees are now trained based on different sequential sets of features, which guarantees more diversity and better performance. Random forest is a variant tree bagging model with additional feature-based bagging.

##### To employ random forest in our click-through prediction project, we use the package from scikit-learn. Similar to the way we implemented the decision tree in the preceding section, we only tweak the max_depth parameter:

In [19]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=100,
                                       criterion='gini', min_samples_split=30, n_jobs=-1)

##### Besides max_depth, min_samples_split, and class_weight, which are important hyperparameters related to a single decision tree, hyperparameters that are related to a random forest (a set of trees) such as n_estimators are also highly recommended:

In [20]:
grid_search = GridSearchCV(random_forest, parameters,
                           n_jobs=-1, cv=3, scoring='roc_auc')
grid_search.fit(X_train_enc, Y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)

{'max_depth': None}
0.7185197803281614


##### Use the model with the optimal parameter None for max_depth (nodes are expanded until another stopping criterion is met) to predict future unseen cases:

In [21]:
random_forest_best = grid_search.best_estimator_
pos_prob = random_forest_best.predict_proba(X_test_enc)[:, 1]
print('The ROC AUC on testing set is: {0:.3f}'.format(roc_auc_score(Y_test, pos_prob)))

The ROC AUC on testing set is: 0.707


##### max_depth: This is the deepest individual tree. It tends to overfit if it is too deep, or to underfit if it is too shallow.
##### min_samples_split: This hyperparameter represents the minimum number of samples required for further splitting at a node. Too small a value tends to cause overfitting, while too large a value is likely to introduce underfitting. 10, 30, and 50 might be good options to start with.
##### max_features: This parameter represents the number of features to consider for each best splitting point search. Typically, for an m-dimensional dataset, (rounded) is a recommended value for max_features. This can be specified as max_features="sqrt" in scikit-learn. Other options include log2, 20% of the original features to 50%.
##### n_estimators: This parameter represents the number of trees considered for majority voting. Generally speaking, the more trees, the better the performance, but more computation time. It is usually set as 100, 200, 500, and so on.

### Tensorflow version

In [ ]:
# >>> n_iter = 20
# >>> n_classes = 2
# >>> n_features = int(X_train_enc.toarray().shape[1])
# >>> n_trees = 10
# >>> max_nodes = 30000

# >>> x = tf.placeholder(tf.float32, shape=[None, n_features])
# >>> y = tf.placeholder(tf.int64, shape=[None])
# >>> hparams = tensor_forest.ForestHParams(num_classes=n_classes,
#  num_features=n_features, num_trees=n_trees,
#  max_nodes=max_nodes, split_after_samples=30).fill()
# >>> forest_graph = tensor_forest.RandomForestGraphs(hparams)

# >>> train_op = forest_graph.training_graph(x, y)
# >>> loss_op = forest_graph.training_loss(x, y)
# >>> infer_op, _, _ = forest_graph.inference_graph(x)
# >>> auc = tf.metrics.auc(tf.cast(y, tf.int64), infer_op[:, 1])[1]

# >>> init_vars = tf.group(tf.global_variables_initializer(),
#           tf.local_variables_initializer(),
#        resources.initialize_resources(resources.shared_resources()))
# >>> sess = tf.Session()
# >>> sess.run(init_vars)

# >>> batch_size = 1000
# >>> import numpy as np
# >>> indices = list(range(n_train))
# >>> def gen_batch(indices):
# ...     np.random.shuffle(indices)
# ...     for batch_i in range(int(n_train / batch_size)):
# ...         batch_index = indices[batch_i*batch_size:
#                                 (batch_i+1)*batch_size]
# ...         yield X_train_enc[batch_index], Y_train[batch_index]

# >>> for i in range(1, n_iter + 1):
# ...     for X_batch, Y_batch in gen_batch(indices):
# ...         _, l = sess.run([train_op, loss_op], feed_dict=
#                            {x: X_batch.toarray(), y: Y_batch})
# ...     acc_train = sess.run(auc, feed_dict=
#                            {x: X_train_enc.toarray(), y: Y_train})
# ...     print('Iteration %i, AUC of ROC on training set: %f' %
#                                                   (i, acc_train))
# ...     acc_test = sess.run(auc, feed_dict=
#                            {x: X_test_enc.toarray(), y: Y_test})
# ...     print("AUC of ROC on testing set:", acc_test)